In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

### train

In [2]:

train_agg = pd.read_csv('./train/train_agg.csv',header=0,sep='\t')
train_log = pd.read_csv('./train/train_log_re_4.csv',header=0,sep='\t')
log_agg = pd.merge(train_agg,train_log,on='USRID',how='left')
train_flg = pd.read_csv('./train/train_flg.csv',header=0,sep='\t')
train_agg_flag = pd.merge(train_flg,log_agg,on='USRID')
train_agg_flag.fillna(0,inplace=True)


In [3]:
train_agg_flag['V4*V7'] = train_agg_flag.V4 * train_agg_flag.V7
train_agg_flag['V4*V17'] = train_agg_flag.V4 * train_agg_flag.V17
train_agg_flag['V5*V7'] = train_agg_flag.V5 * train_agg_flag.V7
train_agg_flag['V5*V17'] = train_agg_flag.V5 * train_agg_flag.V17
train_agg_flag['V5*V30'] = train_agg_flag.V5 * train_agg_flag.V30
train_agg_flag['V7*CC'] = train_agg_flag.V7 * train_agg_flag.CC
train_agg_flag['V1+V7'] = train_agg_flag.V7 + train_agg_flag.V1
train_agg_flag['V6+CC'] = train_agg_flag.V6 + train_agg_flag.CC
train_agg_flag['V6-CC'] = train_agg_flag.V6 - train_agg_flag.CC
train_agg_flag['V7-V28'] = train_agg_flag.V7 - train_agg_flag.V28
# train_agg_flag['V7/CC'] = train_agg_flag.V7 / train_agg_flag.CC
train_agg_flag['V7+CC'] = train_agg_flag.V7 + train_agg_flag.CC
train_agg_flag['V28-V29'] = train_agg_flag.V28 - train_agg_flag.V29
# train_agg_flag['V28/V29'] = train_agg_flag.V28 / train_agg_flag.V29
# train_agg_flag['V6/CC'] = train_agg_flag.V6 / train_agg_flag.CC

In [4]:
def remove_colliner_features(x,threshold):
    y = x['FLAG']
    x = x.drop('FLAG',1)
    corr_matrix = x.corr()
    iters = range(len(corr_matrix.columns) -1)
    drop_cols = []
    for i in iters:
        for j in range(i):
            item = corr_matrix.iloc[j:(j+1),(i+1):(i+2)]
            col = item.columns
            row = item.index
            val = abs(item.values)
            if val > threshold:
                drop_cols.append(col.values[0])
    drops = set(drop_cols)
    x = x.drop(drops,1)
    x['FLAG'] = y
    return x
train_agg_flag = remove_colliner_features(train_agg_flag,0.7)

In [5]:
columnsT = train_agg_flag.drop('FLAG',1).drop('USRID',1).drop('web',1).columns
features_x = train_agg_flag.drop('FLAG',1).drop('USRID',1).drop('web',1).get_values()
features_y = train_agg_flag.FLAG.get_values()

In [6]:
len(columnsT)

80

### test

In [7]:
# test
test_agg = pd.read_csv('./test/test_agg.csv',header=0,sep='\t')
test_log = pd.read_csv('./test/test_log_re_4.csv',header=0,sep='\t')
test_log_agg = pd.merge(test_agg,test_log,on='USRID',how='left')
test_log_agg.fillna(0,inplace=True)
test_log_agg['V4*V7'] = test_log_agg.V4 * test_log_agg.V7
test_log_agg['V4*V17'] = test_log_agg.V4 * test_log_agg.V17
test_log_agg['V5*V7'] = test_log_agg.V5 * test_log_agg.V7
test_log_agg['V5*V17'] = test_log_agg.V5 * test_log_agg.V17
test_log_agg['V5*V30'] = test_log_agg.V5 * test_log_agg.V30
test_log_agg['V7*CC'] = test_log_agg.V7 * test_log_agg.CC
test_log_agg['V1+V7'] = test_log_agg.V7 + test_log_agg.V1
test_log_agg['V6+CC'] = test_log_agg.V6 + test_log_agg.CC
test_log_agg['V6-CC'] = test_log_agg.V6 - test_log_agg.CC
test_log_agg['V7-V28'] = test_log_agg.V7 - test_log_agg.V28
test_log_agg['V7/CC'] = test_log_agg.V7 / test_log_agg.CC
test_log_agg['V7+CC'] = test_log_agg.V7 + test_log_agg.CC
test_log_agg['V28-V29'] = test_log_agg.V28 - test_log_agg.V29
test_log_agg['V28/V29'] = test_log_agg.V28 / test_log_agg.V29
test_log_agg['V6/CC'] = test_log_agg.V6 / test_log_agg.CC
key = test_log_agg.USRID
# test = test_features.get_values()
test = test_log_agg[columnsT].get_values()

### model

In [19]:
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
xx_cv = []
xx_pre = []
N = 3
skf = StratifiedKFold(n_splits=N,shuffle=False,random_state=42)
for train_in,test_in in skf.split(features_x,features_y):
    X_train,X_test,y_train,y_test = features_x[train_in],features_x[test_in],features_y[train_in],features_y[test_in]
    # create dataset for lightgbm
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
    # specify your configurations as a dict
    params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': {'auc'},
        'num_leaves': 15,
        'learning_rate': 0.01,
        'feature_fraction': 0.5,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': 0,
         'max_depth':4,
         'lambda_l1':1.5,
        'lambda_l2':0.0,
        'max_bin':100,
        'scale_pos_weight':20,
    }
    params['min_data_in_leaf'] = 200
    print('Start training...')
    # train
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=2000,
                    valid_sets=lgb_eval,
                    verbose_eval=250,
                    early_stopping_rounds=50)

    # print('Save model...')
    # save model to file
    # gbm.save_model('model.txt')

    print('Start predicting...')
    y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
    xx_cv.append(roc_auc_score(y_test,y_pred))
    xx_pre.append(gbm.predict(test, num_iteration=gbm.best_iteration))

s = 0
for i in xx_pre:
    s = s + i

s = s /N

res = pd.DataFrame()
res['USRID'] = list(key)
res['RST'] = list(s)

print('xx_cv',np.mean(xx_cv))

import time
time_date = time.strftime('%Y-%m-%d',time.localtime(time.time()))
res.to_csv('./submit/%s_%s.csv'%(str(time_date),str(np.mean(xx_cv)).split('.')[1]),index=False,sep='\t')

Start training...
Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.850152
[500]	valid_0's auc: 0.852906
Early stopping, best iteration is:
[599]	valid_0's auc: 0.853207
Start predicting...
Start training...
Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.84475
[500]	valid_0's auc: 0.848905
[750]	valid_0's auc: 0.850709
[1000]	valid_0's auc: 0.85107
Early stopping, best iteration is:
[1005]	valid_0's auc: 0.851106
Start predicting...
Start training...
Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.852796
[500]	valid_0's auc: 0.858028
[750]	valid_0's auc: 0.860059
Early stopping, best iteration is:
[904]	valid_0's auc: 0.860529
Start predicting...
xx_cv 0.8549474997594632


In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
N = 5
skf = StratifiedKFold(n_splits=N,shuffle=False,random_state=42)
from sklearn.metrics import roc_auc_score
xx_cv = []
xx_pre = []
for train_in,test_in in skf.split(features_x,features_y):
    X_train,X_test,y_train,y_test = features_x[train_in],features_x[test_in],features_y[train_in],features_y[test_in]

    rf0 = RandomForestClassifier(min_samples_split=100,min_samples_leaf=50,max_depth=10,bootstrap=False,max_features='sqrt' ,random_state=10,min_weight_fraction_leaf=0,n_estimators=4000)
    rf0.fit(X_train,y_train)
    y_predprob = rf0.predict_proba(X_test)[:,1]
    print('Start predicting...')
    xx_cv.append(roc_auc_score(y_test,y_predprob))
    xx_pre.append(rf0.predict_proba(test)[:, 1])

s = 0
for i in xx_pre:
    s = s + i

s = s /N

res = pd.DataFrame()
res['USRID'] = list(key)
res['RST'] = list(s)

print('xx_cv',np.mean(xx_cv))

import time
time_date = time.strftime('%Y-%m-%d',time.localtime(time.time()))
res.to_csv('./submit/%s_%s.csv'%(str(time_date),str(np.mean(xx_cv)).split('.')[1]),index=False,sep='\t')                                                          

Start predicting...
Start predicting...
Start predicting...
Start predicting...
Start predicting...
xx_cv 0.8481004963347125


In [33]:
from sklearn.grid_search import GridSearchCV
param_test1= {}
gsearch1= GridSearchCV(estimator = RandomForestClassifier(min_samples_split=100,
                                 min_samples_leaf=50,max_depth=10,bootstrap=False,max_features='sqrt' ,random_state=10,min_weight_fraction_leaf=0,n_estimators=4000),
                       param_grid =param_test1, scoring='roc_auc',cv=3)
gsearch1.fit(features_x,features_y)
gsearch1.grid_scores_,gsearch1.best_params_, gsearch1.best_score_

([mean: 0.84811, std: 0.00386, params: {}], {}, 0.8481078553184083)

In [ ]:
gbm.feature_importance()
train_agg_flag.drop('FLAG',1).drop('USRID',1).columns
def getIm(score,col):
    if len(score) != len(col):
        print('err')
    for i in range(len(score)):
        if score[i]>250:
            pass
        print('score:{0}, col:{1}'.format(score[i],col[i]))
getIm(cat_model.feature_importances_,train_agg_flag.drop('FLAG',1).drop('USRID',1).drop('web',1).columns)